In [ ]:
import mysql.connector
from mysql.connector import Error
from typing import Optional, List, Dict

In [ ]:
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "",
}
DB_NAME = "university"

In [ ]:
def create_database(db_name: str = DB_NAME):
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        print(f"База данных '{db_name}' создана или уже существует")
    except Error as e:
        print(f"Ошибка создания базы данных: {e}")
    finally:
        conn.close()

create_database()

In [ ]:
def get_connection():
    return mysql.connector.connect(database=DB_NAME, **DB_CONFIG)

### task 1
___

In [ ]:
def create_tables():
    try:
        conn = get_connection()
        cursor = conn.cursor()
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS students (
            id INT AUTO_INCREMENT PRIMARY KEY,
            first_name VARCHAR(100) NOT NULL,
            last_name VARCHAR(100) NOT NULL,
            group_name VARCHAR(50) NOT NULL,
            admission_year INT NOT NULL,
            average_grade FLOAT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS courses (
            id INT AUTO_INCREMENT PRIMARY KEY,
            course_name VARCHAR(255) UNIQUE NOT NULL,
            instructor VARCHAR(255) NOT NULL,
            credits INT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS student_courses (
            student_id INT,
            course_id INT,
            PRIMARY KEY(student_id, course_id),
            FOREIGN KEY(student_id) REFERENCES students(id) ON DELETE CASCADE,
            FOREIGN KEY(course_id) REFERENCES courses(id) ON DELETE CASCADE
        )
        """)
        
        conn.commit()
        print("Таблицы созданы")
    except Error as e:
        print(f"Ошибка создания таблиц: {e}")
    finally:
        conn.close()

create_tables()

### task 2
___

In [ ]:
def add_student(first_name, last_name, group_name, admission_year, average_grade=None) -> Optional[int]:
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO students (first_name, last_name, group_name, admission_year, average_grade) VALUES (%s,%s,%s,%s,%s)",
            (first_name, last_name, group_name, admission_year, average_grade)
        )
        conn.commit()
        return cursor.lastrowid
    except Error as e:
        print(f"Ошибка добавления студента: {e}")
        return None
    finally:
        conn.close()

In [ ]:
def get_all_students() -> List[Dict]:
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT * FROM students")
    rows = cursor.fetchall()
    conn.close()
    return rows

In [ ]:
def get_students_by_group(group_name: str) -> List[Dict]:
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT * FROM students WHERE group_name = %s", (group_name,))
    rows = cursor.fetchall()
    conn.close()
    return rows

In [ ]:
def update_student_grade(student_id: int, new_grade: float) -> bool:
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE students SET average_grade = %s WHERE id = %s", (new_grade, student_id))
    conn.commit()
    updated = cursor.rowcount > 0
    conn.close()
    return updated

In [ ]:
def delete_student(student_id: int) -> bool:
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM students WHERE id = %s", (student_id,))
    conn.commit()
    deleted = cursor.rowcount > 0
    conn.close()
    return deleted

### task 3
___

In [ ]:
def add_course(course_name, instructor, credits) -> Optional[int]:
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO courses (course_name, instructor, credits) VALUES (%s,%s,%s)",
                        (course_name, instructor, credits))
        conn.commit()
        return cursor.lastrowid
    except Error as e:
        print(f"Ошибка добавления курса: {e}")
        return None
    finally:
        conn.close()

In [ ]:
def enroll_student_in_course(student_id, course_id) -> bool:
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO student_courses (student_id, course_id) VALUES (%s,%s)", (student_id, course_id))
        conn.commit()
        return True
    except Error as e:
        print(f"Ошибка зачисления студента: {e}")
        return False
    finally:
        conn.close()

In [ ]:
def get_student_courses(student_id) -> List[Dict]:
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("""
        SELECT c.* FROM courses c
        JOIN student_courses sc ON c.id = sc.course_id
        WHERE sc.student_id = %s
    """, (student_id,))
    rows = cursor.fetchall()
    conn.close()
    return rows

In [ ]:
def get_course_students(course_id) -> List[Dict]:
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("""
        SELECT s.* FROM students s
        JOIN student_courses sc ON s.id = sc.student_id
        WHERE sc.course_id = %s
    """, (course_id,))
    rows = cursor.fetchall()
    conn.close()
    return rows

In [ ]:
def transfer_student(student_id: int, new_group: str) -> bool:
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("START TRANSACTION")
        
        cursor.execute("SELECT id FROM students WHERE id = %s", (student_id,))
        if cursor.fetchone() is None:
            raise ValueError("Студент не найден")
        
        cursor.execute("UPDATE students SET group_name=%s WHERE id=%s", (new_group, student_id))
        cursor.execute("DELETE FROM student_courses WHERE student_id=%s", (student_id,))
        
        conn.commit()
        return True
    except Exception as e:
        conn.rollback()
        print(f"Ошибка перевода студента: {e}")
        return False
    finally:
        conn.close()

In [ ]:
s1 = add_student("Иван", "Петров", "ГР-01", 2023, 4.5)
s2 = add_student("Мария", "Иванова", "ГР-01", 2023, 4.8)
s3 = add_student("Алексей", "Сидоров", "ГР-02", 2023, 3.9)

c1 = add_course("Математика", "Проф. Иванов", 5)
c2 = add_course("Физика", "Проф. Петрова", 4)
c3 = add_course("Программирование", "Доц. Сидоров", 6)

enroll_student_in_course(s1, c1)
enroll_student_in_course(s1, c3)
enroll_student_in_course(s3, c2)

print("Все студенты:", get_all_students())
print("Курсы Ивана:", get_student_courses(s1))

transfer_student(s1, "ГР-03")
print("После перевода студента:", get_all_students())

### task 4
____

In [1]:
from uni_bd import UniversityDB

with UniversityDB() as db:
    stats = db.get_student_statistics()
    print(stats)

{'total_students': 0, 'average_grade': None, 'groups': []}
